# IndicePA Crawling


Questo notebook arricchisce i dati Anac con informazioni presenti negli open data di IndicePA

IndicePA ragruppa le Pubbliche Amministrazioni (PA) in 3 categorie:

1. *AMMINISTRAZIONI*: enti pubblici;

2. *AOO*: Aree Organizzative Omogenee;

3. *OU*: Unità Organizzative

Per ogni categoria, IndicePA associa differenti informazioni. Per maggiori dettagli vedi [qui](http://www.indicepa.gov.it/public-services/docs-read-service.php?dstype=FS&filename=Metadati_Open_Data.pdf)

In particolare, il seguente notebook estrae dal file *lista_pa.json* tutti i codici fiscali (cf) delle PA censite da Anac. Ogni cf viene quindi ricercato nei file *data/indicePA/serv_fatt.txt* (PEC-CECPAC)  e *data/indicePA/amministrazioni.txt* (opendata contenente informazioni su tutte le PA di tipo amministrazioni) e vengono estratti i seguenti attributi:

* Codice Fiscale
* Regione
* Provincia
* Comune 
* Indirizzo
* Tipologia Istat
* tipologia_amm

Il file *lista_pa.json* è generato dalla seguente chiamata rest:
> curl -H "Content-Type: application/json" -X POST -d  '{"anno":"2018","codiceFiscaleAmministrazione":"","denominazioneAmministrazione":"","identificativoComunicazione":""}' https://dati.anticorruzione.it/rest/legge190/ricerca\?max\=200000\&start\=0 -o lista_pa.json

## Note

1. Ci sono 1118 Codici Fiscali censiti da Anac che non son presenti in IndicePA. Nella maggior parte dei casi si tratta di SPA (e.g. 10200211000, 00222610289, 95107460107, etc.).

2. Ci sono 4 PA i cui Codici Fiscali non corripondono con nessun *codice amministrazione* (i.e. 00491510822, 00251300067, 01607380670, 00431990068). 

In [2]:
import pandas as pd
import numpy
import csv

In [2]:
file_amministrazioni = "data/indicePA/amministrazioni.txt"
file_aoo = "data/indicePA/aoo.txt"
file_ou = "data/indicePA/ou.txt"
file_PEC_CECPAC = "data/indicePA/pec.txt"
file_serv_fatt = "data/indicePA/serv_fatt.txt"
file_lista_anac_pa = "data/anac/lista_pa.json"

In [3]:
#df_amm = pd.read_csv(file_amministrazioni, sep="\t", index_col="cod_amm", dtype=object)
df_amm = pd.read_csv(file_amministrazioni, sep="\t", dtype=object)
df_amm = df_amm[[ 'cod_amm','Cf','Comune', 'Provincia', 'Regione', 'Indirizzo', 'tipologia_istat', 'tipologia_amm', 'acronimo']]
df_amm.head()


,cod_amm,Cf,Comune,Provincia,Regione,Indirizzo,tipologia_istat,tipologia_amm,acronimo
0,054,03301860544,Perugia,PG,Umbria,"Via Guerriero Guerra, 21",Aziende Sanitarie Locali,Pubbliche Amministrazioni,
1,055,01499590550,Terni,TR,Umbria,"Viale Donato Bramante, 37",Aziende Sanitarie Locali,Pubbliche Amministrazioni,ASLUMBRIA2
2,056,01455570562,Viterbo,VT,Lazio,"Via Enrico Fermi, 15",Aziende Sanitarie Locali,Pubbliche Amministrazioni,NaN
3,058,04733471009,Tivoli,RM,Lazio,Via Acquaregna 1/15,Aziende Sanitarie Locali,Pubbliche Amministrazioni,NaN
4,080,02638720801,Reggio Calabria,RC,Calabria,"Via Sant'Anna II Tronco, 18/p",Aziende Sanitarie Locali,Pubbliche Amministrazioni,ASPRC


In [4]:
grouped_amm = df_amm[["tipologia_istat", "tipologia_amm", "Cf"]].groupby(["tipologia_istat", "tipologia_amm"]).agg(["count"])
grouped_amm.head()

,,Cf
,,count
tipologia_istat,tipologia_amm,
Agenzie Fiscali,Pubbliche Amministrazioni,4
Agenzie Regionali Sanitarie,Pubbliche Amministrazioni,12
Agenzie Regionali e Provinciale per la Rappresentanza Negoziale,Pubbliche Amministrazioni,1
Agenzie Regionali per le Erogazioni in Agricoltura,Pubbliche Amministrazioni,9
Agenzie ed Enti Regionali del Lavoro,Pubbliche Amministrazioni,9


In [5]:
df_aoo = pd.read_csv(file_aoo, sep="\t")
print(df_aoo.columns)
df_aoo = df_aoo[['cod_amm', 'cod_aoo', 'Comune', 'Provincia', 'Regione', 'Indirizzo']]
df_aoo.head()

Index(['cod_amm', 'cod_aoo', 'des_aoo', 'dat_istituz', 'Comune', 'Cap',
       'Provincia', 'Regione', 'Indirizzo', 'Tel', 'nome_resp', 'cogn_resp',
       'mail_resp', 'tel_resp', 'Fax', 'mail1', 'tipo_mail1', 'mail2',
       'tipo_mail2', 'mail3', 'tipo_mail3'],
      dtype='object')


,cod_amm,cod_aoo,Comune,Provincia,Regione,Indirizzo
0,055,ASLUMBRIA2,Terni,TR,Umbria,V.le D. Bramante n. 37
1,058,ABS,Tivoli,RM,Lazio,Via Acquaregna 1/15
2,058,DIP-Prevenzione,Tivoli,RM,Lazio,Via Acquaregna 1/15
3,092,AOO-AOB,Cagliari,CA,Sardegna,"Piazzale A. Ricchi, 1"
4,093,protgen,Pordenone,PN,Friuli Venezia Giulia,"Via Vecchia Ceramica, 1"


In [6]:
df_ou = pd.read_csv(file_ou, sep="\t")
print(df_ou.columns)
df_ou = df_ou[['cod_amm', 'cod_aoo', 'cod_ou', 'comune', 'provincia', 'Regione', 'Indirizzo']]

df_ou = df_ou.rename(columns={'comune': 'Comune', 'provincia': 'Provincia'})
df_ou.head()

Index(['cod_ou', 'cod_aoo', 'des_ou', 'comune', 'Cap', 'provincia', 'Regione',
       'Indirizzo', 'Tel', 'nome_resp', 'cogn_resp', 'mail_resp', 'tel_resp',
       'cod_amm', 'cod_ou_padre', 'Fax', 'cod_uni_ou', 'mail1', 'tipo_mail1',
       'mail2', 'tipo_mail2', 'mail3', 'tipo_mail3'],
      dtype='object')


/Users/fabiana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cod_amm,cod_aoo,cod_ou,Comune,Provincia,Regione,Indirizzo
0,c_f335,aooac,serfin,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
1,c_f335,aooac,sertec,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
2,c_f335,aooac,comlav,Moncalieri,TO,Piemonte,Piazza Vittorio Emanuele II snc
3,avvst,adsrm,adsrm01,Roma,RM,Lazio,"Via dei Portoghesi, 12"
4,avvst,adsrm,adsrm03,Roma,RM,Lazio,"Via del Clementino, 91/A Roma"


In [7]:
#df_pec = pd.read_csv(file_PEC_CECPAC, sep="\t", index_col="cod_amm")
df_pec = pd.read_csv(file_PEC_CECPAC, sep="\t")
print(df_pec.columns)
df_pec = df_pec[['cod_amm','Tipo', 'Tipologia Istat']]
df_pec.head()

Index(['cod_amm', 'Descrizione', 'Tipo', 'Tipologia Istat', 'Regione',
       'Provincia', 'Comune', 'Mail', 'Tipo Mail'],
      dtype='object')


,cod_amm,Tipo,Tipologia Istat
0,abam_043,SERVAMM,"Istituzioni per l'Alta Formazione Artistica, M..."
1,abfitlpb,SERVAMM,Autorita' di Bacino
2,abspa,SERVAMM,Gestori di Pubblici Servizi
3,ac_va,SERVAMM,Automobile Club Federati ACI
4,aimcr,SERVAMM,Aziende Pubbliche di Servizi alla Persona


In [8]:
df_serv_fatt = pd.read_csv(file_serv_fatt, sep="\t", dtype=object)
print(df_serv_fatt.columns)
#df_serv_fatt = df_serv_fatt[[ 'cod_amm','Cf','Comune', 'Provincia', 'Regione', 'Indirizzo']]
df_serv_fatt = df_serv_fatt.rename(columns={'cf': 'Cf'})
df_serv_fatt.head()

Index(['cod_amm', 'cod_ou', 'cod_uni_ou', 'des_ou', 'regione', 'provincia',
       'comune', 'indirizzo', 'cap', 'cf', 'dt_verifica_cf', 'data_avvio_sfe'],
      dtype='object')


,cod_amm,cod_ou,cod_uni_ou,des_ou,regione,provincia,comune,indirizzo,cap,Cf,dt_verifica_cf,data_avvio_sfe
0,054,Uff_eFatturaPA,UF9FAJ,Uff_eFatturaPA,Umbria,PG,Perugia,"Via Guerriero Guerra, 21",06127,03301860544,2015-04-01,2015-03-31
1,055,SS01,Y0BN4H,Servizio economico finanziario Sanitario,Umbria,TR,Terni,Via Bramante 37,05100,01499590550,2015-04-02,2015-03-31
2,055,SS02,H2J28R,Servizio economico finanziario Servizi Sociali,Umbria,TR,Terni,Via Bramante 37,05100,01499590550,2015-03-31,2015-03-31
3,055,Uff_eFatturaPA,UF7K0M,Uff_eFatturaPA,Umbria,TR,Terni,"Viale Donato Bramante, 37",05100,01499590550,2015-03-31,2015-03-31
4,056,Uff_eFatturaPA,UF9IAK,Uff_eFatturaPA,Lazio,VT,Viterbo,"Via Enrico Fermi, 15",01100,01455570562,2015-04-08,2015-03-31


In [9]:
print("Dimensione file amministrazioni.txt", df_amm.shape)
print("Dimensione file aoo.txt", df_aoo.shape)
print("Dimensione file ou.txt", df_ou.shape)
print("Dimensione file pec.txt", df_pec.shape)

Dimensione file amministrazioni.txt (22766, 9)
Dimensione file aoo.txt (26810, 6)
Dimensione file ou.txt (92854, 7)
Dimensione file pec.txt (112754, 3)


In [10]:
import json
from pprint import pprint
cf_set_serv_fatt = set(df_serv_fatt.Cf.values)
cf_set_amm = set(df_amm.Cf.values)
data = json.load(open(file_lista_anac_pa))

f_indice_pa = open("data/indicePA/indicePA.tsv", 'w')
f_other = open("data/indicePA/other_pa.tsv", 'w')
writer_indice_pa = csv.writer(f_indice_pa, delimiter ='\t')
writer_other_pa = csv.writer(f_other, delimiter ='\t')

count = 0

res = ["cf", "cod_amm", "regione", "provincia", "comune", "indirizzo", "tipologia_istat", "tipologia_amm"]
writer_indice_pa.writerow(res)

77

In [26]:
for pa in data["result"]:
    esito = pa["esitoUltimoTentativoAccessoUrl"]
    if (esito == "successo"):
        cf = pa["codiceFiscale"]
        
        if(cf in cf_set_amm):
            try:
                cod_amm = df_amm.loc[df_amm['Cf'] == cf].iloc[0]['cod_amm']
                take0 = df_amm.loc[df_amm['cod_amm'] == cod_amm].iloc[0]
                regione = take0['Regione'].replace("\t", "")
                provincia = str(take0['Provincia']).replace("\t", "")
                comune = take0['Comune'].replace("\t", "")
                indirizzo = take0['Indirizzo'].replace("\t", "")
                tipologia_istat = take0['tipologia_istat'].replace("\t", "")
                tipologia_amm = take0['tipologia_amm'].replace("\t", "")
                
                res = [cf, cod_amm, regione, provincia, comune, indirizzo, tipologia_istat, tipologia_amm]
                writer_indice_pa.writerow(res)
            except: # catch *all* exceptions
                print("CF in df_amm",cf)
        elif(cf in cf_set_serv_fatt):
            try:
                cod_amm = df_serv_fatt.loc[df_serv_fatt['Cf'] == cf].iloc[0]['cod_amm']
                take0 = df_amm.loc[df_amm['cod_amm'] == cod_amm].iloc[0]
                regione = take0['Regione'].replace("\t", "")
                provincia = str(take0['Provincia']).replace("\t", "")
                comune = take0['Comune'].replace("\t", "")
                indirizzo = take0['Indirizzo'].replace("\t", "")
                tipologia_istat = take0['tipologia_istat'].replace("\t", "")
                tipologia_amm = take0["tipologia_amm"].replace("\t", "")
                res = [cf, cod_amm, regione, provincia, comune, indirizzo, tipologia_istat, tipologia_amm]
                writer_indice_pa.writerow(res)
            except: # catch *all* exceptions
                #e = sys.exc_info()[0]
                print("CF in df_serv_fatt",cf)
        else:
            #print(cf, " is not present")
            count = count + 1
            writer_other_pa.writerow([cf])
            #if(count % 100 == 0):
                #print(cf)

f_indice_pa.close
f_other.close
print("Totale cf non presenti in IndicePA: ", count)

Totale cf non presenti in IndicePA:  1118


## Arricchimento Dataframe Anac con info su *Struttura Proponente*

Il precedente codice permette di estrarre informazioni su strutture proponenti presenti nel json etratto precedentemente

In [3]:
f_anac = "data/anac/anacDataset_1522836870094_postprocessed2.tsv"
#df = pd.read_csv(f_anac, sep="\t", error_bad_lines=False, parse_dates=True)
df = pd.read_csv(f_anac, sep="\t", parse_dates=True)

/Users/fabiana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df.cfStrutturaProponente = df.cfStrutturaProponente.astype(str)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3913440 entries, 0 to 3913439
Data columns (total 17 columns):
cig                                  object
cfStrutturaProponente                object
denominazioneStrutturaProponente     object
oggetto                              object
sceltaContraente                     object
importoAggiudicazione                float64
importoSommeLiquidate                float64
dataInizio                           object
dataUltimazione                      object
jsonPartecipanti                     object
jsonAggiudicatari                    object
totalePartecipanti                   int64
totaleAggiudicatari                  int64
totaleRaggruppamentopartecipanti     int64
totaleRaggruppamentoAggiudicatari    int64
cfPrimoaggiudicatario                object
denominazionePrimoaggiudicatario     object
dtypes: float64(2), int64(4), object(11)
memory usage: 507.6+ MB


In [13]:
for cf in set(df.cfStrutturaProponente):
    

{'80012870475',
 '91027300655',
 '80001970252',
 '95146530654',
 '92041030880',
 '605360636',
 '80006930806',
 '83002370696',
 '81000550640',
 '2924070150',
 '80044030106',
 '81001910223',
 '00848610176',
 '00085210896',
 '454560046',
 '2155050343',
 '90035700096',
 '82008730408',
 '95170090633',
 '97714270580',
 '394620785',
 '80001180696',
 '703920132',
 '81000770305',
 '105180145',
 '93061260712',
 '80003610567',
 '90146880738',
 '80024900211',
 '95146410659',
 '2606400832',
 '80050670944',
 '440760545',
 '80081870372',
 '80003090745',
 '94045460279',
 '444270136',
 '90038780442',
 '81000370924',
 '538350026',
 '1301430714',
 '80009950223',
 '1991340017',
 '263320228',
 '987750502',
 '260520127',
 '91111780689',
 '82011770631',
 '82003030655',
 '80009550031',
 '443770541',
 '353510357',
 '81001210228',
 '84004990150',
 '90013600359',
 '86000390830',
 '82010270633',
 '95160180105',
 '627060361',
 '92023440263',
 '365990712',
 '01266270535',
 '321280273',
 '81001650548',
 '80002070763

## Link Utili

[1] [Specifiche ANAC](https://www.anticorruzione.it/portal/rest/jcr/repository/collaboration/Digital%20Assets/anacdocs/Servizi/ServiziOnline/AdempimentoLegge190/Specifiche%20Tecniche%20Legge%20190%20v1.2_finale.pdf)

[2] [](http://portaletrasparenza.anticorruzione.it/Microstrategy/asp/Main.aspx)

[3] [Open data IndicePA](http://www.indicepa.gov.it/documentale/n-opendata.php)